In [2]:
import numpy as np
import pandas as pd

import os

In [3]:
data_folder = '../data/routes_short_2023-05-05|22:07:50/'

In [4]:
scenario_header = ["cloudiness",
                   "precipitation",
                   "precipitation_deposits",
                   "wind_intensity",
                   "sun_azimuth_angle",
                   "sun_altitude_angle",
                   "fog_density",
                   "wetness",
                   "fog_falloff",
                   "vehicle_infront", 
                   "vehicle_opposite", 
                   "vehicle_side",
                   "start_offset",
                   "end_offset"]
scenarios = pd.read_csv(data_folder+'scenario.csv',names=scenario_header)
print(scenarios.shape)

(439, 14)


In [5]:
criterion_header = ["RouteCompletionTest",   
                    "RouteCompletionTest_figure",
                    "OutsideRouteLanesTest", 
                    "OutsideRouteLanesTest_figure",
                    "CollisionTest",         
                    "CollisionTest_figure",
                    "RunningRedLightTest",   
                    "RunningRedLightTest_figure",
                    "RunningStopTest",       
                    "RunningStopTest_figure",
                    "InRouteTest", 
                    "InRouteTest_figure",          
                    "AgentBlockedTest",
                    "AgentBlockedTest_figure",      
                    "Timeout"]
criterions = pd.read_csv(data_folder+'criterion-replace.csv',names=criterion_header)
print(criterions.shape)

(439, 15)


In [6]:
scenario_criterion = pd.concat([scenarios, criterions[["RouteCompletionTest", "CollisionTest", "RunningRedLightTest", "Timeout"]]],axis=1)

In [9]:
scenario_criterion

,cloudiness,precipitation,precipitation_deposits,wind_intensity,sun_azimuth_angle,sun_altitude_angle,fog_density,wetness,fog_falloff,vehicle_infront,vehicle_opposite,vehicle_side,start_offset,end_offset,RouteCompletionTest,CollisionTest,RunningRedLightTest,Timeout
0,0.071136,0.273546,0.655469,0.198134,0.720386,0.244049,0.676710,0.759941,0.051406,0.988784,0.226244,0.204948,0.784768,0.202398,0,0,0,0
1,0.375515,0.653659,0.719721,0.147626,0.557042,0.865031,0.554341,0.738864,0.485065,0.361723,0.966843,0.116823,0.050662,0.037090,0,0,0,0
2,0.396908,0.266277,0.175777,0.231517,0.410776,0.285436,0.015449,0.404476,0.056316,0.605676,0.824694,0.105960,0.330345,0.480143,0,0,0,0
3,0.308592,0.509314,0.159497,0.360004,0.335843,0.021912,0.100606,0.093278,0.127418,0.212385,0.064637,0.058994,0.487564,0.642052,0,0,0,0
4,0.847700,0.696302,0.407896,0.490933,0.949995,0.158169,0.740556,0.950721,0.892074,0.930402,0.674357,0.539991,0.044751,0.931640,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434,0.737023,0.488781,0.430072,0.057317,0.195883,0.833937,0.678136,0.663629,0.506084,0.069737,0.331663,0.555475,0.560858,0.307836,0,0,0,0
435,0.691164,0.895655,0.897923,0.581900,0.842071,0.130811,0.480834,0.660447,0.281690,0.923852,0.433620,0.392038,0.780196,0.891311,1,0,0,1
436,0.793815,0.683412,0.002570,0.744070,0.809537,0.859068,0.808822,0.811194,0.439028,0.273048,0.075028,0.656146,0.102465,0.664017,1,0,0,1
437,0.022925,0.815955,0.129537,0.648136,0.594590,0.051505,0.647217,0.345200,0.792262,0.865359,0.826071,0.050037,0.257980,0.422620,0,0,0,0


In [28]:
print(scenario_criterion.columns.tolist()[:14])
scenario_vectors = scenario_criterion[scenario_criterion.columns.tolist()[:14]].to_numpy()
print(scenario_vectors.shape)
print(scenario_vectors[:5])


['cloudiness', 'precipitation', 'precipitation_deposits', 'wind_intensity', 'sun_azimuth_angle', 'sun_altitude_angle', 'fog_density', 'wetness', 'fog_falloff', 'vehicle_infront', 'vehicle_opposite', 'vehicle_side', 'start_offset', 'end_offset']
(439, 14)
[[0.07113632 0.2735465  0.65546894 0.19813449 0.72038609 0.24404919
  0.67670974 0.75994097 0.05140624 0.98878404 0.22624446 0.20494811
  0.78476757 0.20239828]
 [0.37551536 0.65365905 0.71972051 0.14762641 0.55704217 0.8650307
  0.55434093 0.73886421 0.4850653  0.36172333 0.96684321 0.11682335
  0.05066178 0.03709034]
 [0.39690791 0.26627747 0.17577719 0.23151658 0.41077626 0.28543616
  0.01544943 0.40447605 0.05631604 0.60567591 0.82469433 0.10595996
  0.33034534 0.48014322]
 [0.30859197 0.50931421 0.15949709 0.36000441 0.3358433  0.02191171
  0.10060637 0.0932778  0.12741767 0.21238524 0.06463658 0.0589939
  0.48756396 0.64205165]
 [0.84769988 0.69630182 0.40789604 0.49093334 0.94999465 0.15816911
  0.740556   0.95072069 0.89207437 

In [29]:
print(scenario_criterion.columns.tolist()[14:])
fitness_results = scenario_criterion[scenario_criterion.columns.tolist()[14:]].to_numpy()
fitness_results[:,0] = 1-fitness_results[:,0]
print(fitness_results.sum(axis=0))
print(fitness_results.sum())
fitness_results[:5]

['RouteCompletionTest', 'CollisionTest', 'RunningRedLightTest', 'Timeout']
[316   4  33  86]
439


array([[1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0]])

In [171]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt


class FitnessDataset(Dataset):
    def __init__(self, scenario_vectors, fitness_results):
        self.data = torch.from_numpy(scenario_vectors).float()
        self.label = torch.from_numpy(fitness_results).float()
    
    def __len__(self):
        return (len(self.label))
    
    def __getitem__(self, index):
        return self.data[index], self.label[index] 

In [172]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(scenario_vectors, fitness_results, test_size=0.33, random_state=14159)
train_data = FitnessDataset(X_train, y_train)
test_data = FitnessDataset(X_test, y_test)

In [208]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_data, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=16, shuffle=True)

In [209]:
# Display image and label.
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
label = train_labels[0]
print(f"Label: {label}")

Feature batch shape: torch.Size([16, 14])
Labels batch shape: torch.Size([16, 4])
Label: tensor([1., 0., 0., 0.])


In [219]:
import numpy as np

import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class MLP(nn.Module):

    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(14, 32)
        # self.fc2 = nn.Linear(32, 64)
        # self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 4)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        # x = F.relu(self.fc2(x))
        # x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [216]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            print(pred.argmax(axis=1))
            print(y.argmax(axis=1))
            print()
            correct += (pred.argmax(axis=1) == y.argmax(axis=1)).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [220]:
mlp = MLP()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(mlp.parameters(), lr=0.001, momentum=0.9)

epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, mlp, criterion, optimizer)
    test_loop(test_dataloader, mlp, criterion)
print("Done!")

Epoch 1
-------------------------------
loss: 1.360244  [   16/  294]
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 3, 0, 3, 0, 0, 0, 3, 0, 0, 0, 0, 2, 3, 0])

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([3, 3, 0, 0, 0, 1, 3, 0, 3, 0, 0, 0, 0, 0, 0, 3])

tensor([0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0])

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([3, 0, 0, 0, 3, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 3])

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 2, 2, 3, 3, 0, 0, 0, 3, 0, 3, 0, 0, 0, 0])

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 2, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0])

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 3, 0, 2, 0, 3, 0, 0, 0, 3, 3, 0, 0, 0, 0, 0])

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 3, 2, 3, 0, 0, 2, 0, 0, 0, 0, 3, 0, 0, 0, 0])

tensor([0,